In [1]:
! pip install geemap

In [2]:
import geemap

In [3]:
map = geemap.Map(center=(26.293805525369162, 77.54585509702882), zoom=5)
map

Map(center=[26.293805525369162, 77.54585509702882], controls=(WidgetControl(options=['position', 'transparent_…

In [4]:
map.add_basemap('SATELLITE')

#map.set_center(30.6302, 76.7248)
#map.set_center(30.631446509381377, 76.72447156832993)


In [5]:

# Go to Plaksha
map.set_center(76.72447156832993, 30.631446509381377 , zoom=16)
#map.set_center(76.7248, 30.6302)
#map.set_zoom(8)

In [6]:
basemaps = geemap.basemaps

for basemap in basemaps:
    print(basemap)

OpenStreetMap
Esri.WorldStreetMap
Esri.WorldImagery
Esri.WorldTopoMap
FWS NWI Wetlands
FWS NWI Wetlands Raster
NLCD 2021 CONUS Land Cover
NLCD 2019 CONUS Land Cover
NLCD 2016 CONUS Land Cover
NLCD 2013 CONUS Land Cover
NLCD 2011 CONUS Land Cover
NLCD 2008 CONUS Land Cover
NLCD 2006 CONUS Land Cover
NLCD 2004 CONUS Land Cover
NLCD 2001 CONUS Land Cover
USGS NAIP Imagery
USGS NAIP Imagery False Color
USGS NAIP Imagery NDVI
USGS Hydrography
USGS 3DEP Elevation
ESA Worldcover 2020
ESA Worldcover 2020 S2 FCC
ESA Worldcover 2020 S2 TCC
ESA Worldcover 2021
ESA Worldcover 2021 S2 FCC
ESA Worldcover 2021 S2 TCC
BaseMapDE.Color
BaseMapDE.Grey
BasemapAT.basemap
BasemapAT.grau
BasemapAT.highdpi
BasemapAT.orthofoto
BasemapAT.overlay
BasemapAT.surface
BasemapAT.terrain
CartoDB.DarkMatter
CartoDB.DarkMatterNoLabels
CartoDB.DarkMatterOnlyLabels
CartoDB.Positron
CartoDB.PositronNoLabels
CartoDB.PositronOnlyLabels
CartoDB.Voyager
CartoDB.VoyagerLabelsUnder
CartoDB.VoyagerNoLabels
CartoDB.VoyagerOnlyLabe

In [7]:
# Add the hybrid layer (satellite + labels)
map.add_tile_layer(
    url="http://mt1.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}",
    name="Google Hybrid",
    attribution="Google"
)


In [8]:
import ee
#import geemap.core as geemap

Let us now try to retreive and visualize the satellite imagery for LandSat around the Plaksha region. Here's the Earth Engine Catalog [link](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2?hl=en#colab-python). Notice the cool roof on Infosys building.

In [9]:
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(
    '2021-05-01', '2021-06-01'
)


# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )


dataset = dataset.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

map.add_layer(dataset, visualization, 'True Color (432)')


This is great but probably a few too many pixels. What's the latest real-time medium resolution image we can get? Sentinel 2. Let's try to retreive this.

In [10]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2025-01-01', '2025-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

map.add_layer(dataset.mean(), visualization, 'RGB')

In [11]:
import geopandas as gpd
import geemap
import json

# Load the shapefile
shapefile = 'data/potential-construction.shp'
gdf = gpd.read_file(shapefile)

# Convert the GeoDataFrame to GeoJSON format
geojson = gdf.to_json()

# Parse the GeoJSON string to a dictionary
geojson_dict = json.loads(geojson)

# Convert the GeoJSON to an Earth Engine object
ee_object = geemap.geojson_to_ee(geojson_dict)

# Add the EE object to the map
map.addLayer(ee_object, {}, 'Potential Construction Sites')


In [12]:
dem = ee.Image('USGS/SRTMGL1_003')

vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

map.addLayer(dem, vis_params, 'SRTM DEM', True, 1)
